In [1]:
import pandas as pd 
import random
import numpy as np
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,Flatten, Lambda
from keras.layers import Conv2D, Activation,AveragePooling2D,MaxPooling2D
from keras.optimizers import RMSprop
from keras import backend as K

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('MD16_CapValue_SecondaryCities_29_11_2022.xlsx - Feuil1.csv', sep=',')


In [3]:
headers = pd.rea

AttributeError: module 'pandas' has no attribute 'rea'

In [3]:
df_meta_data = df.iloc[:,3:19]

In [4]:
df_meta_data

,1,2,3,4,5,6,7,81,82,83,9,10,11,12,13,14
0,12,23.0,31.0,41,52,NaN,73.0,811,821,831,92,102,111,122.0,131.0,141
1,12,NaN,31.0,41,52,62.0,72.0,811,821,831,92,102,112,122.0,131.0,141
2,12,NaN,33.0,41,52,62.0,72.0,812,821,832,91,102,112,122.0,131.0,141
3,12,23.0,34.0,43,52,62.0,72.0,812,821,831,91,102,112,121.0,131.0,141
4,13,NaN,32.0,42,54,63.0,71.0,812,821,831,91,102,111,121.0,131.0,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,12,NaN,33.0,41,52,62.0,71.0,811,821,831,93,102,112,121.0,131.0,141
110,12,NaN,34.0,41,52,61.0,71.0,811,821,831,93,102,112,121.0,131.0,141
111,12,NaN,33.0,41,52,62.0,72.0,812,821,831,93,102,111,NaN,131.0,141
112,12,23.0,33.0,41,52,61.0,72.0,811,821,831,93,102,113,121.0,131.0,141


In [5]:
df_meta_data.fillna(df_meta_data.mode().loc[0], inplace=True)

In [6]:
df_meta_data

,1,2,3,4,5,6,7,81,82,83,9,10,11,12,13,14
0,12,23.0,31.0,41,52,62.0,73.0,811,821,831,92,102,111,122.0,131.0,141
1,12,23.0,31.0,41,52,62.0,72.0,811,821,831,92,102,112,122.0,131.0,141
2,12,23.0,33.0,41,52,62.0,72.0,812,821,832,91,102,112,122.0,131.0,141
3,12,23.0,34.0,43,52,62.0,72.0,812,821,831,91,102,112,121.0,131.0,141
4,13,23.0,32.0,42,54,63.0,71.0,812,821,831,91,102,111,121.0,131.0,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,12,23.0,33.0,41,52,62.0,71.0,811,821,831,93,102,112,121.0,131.0,141
110,12,23.0,34.0,41,52,61.0,71.0,811,821,831,93,102,112,121.0,131.0,141
111,12,23.0,33.0,41,52,62.0,72.0,812,821,831,93,102,111,121.0,131.0,141
112,12,23.0,33.0,41,52,61.0,72.0,811,821,831,93,102,113,121.0,131.0,141


In [7]:
import itertools

In [8]:
pairs = list(itertools.combinations(df_meta_data.index, 2))

In [9]:
X_pair = []
y = []

for pair in pairs:
    asset1 = pair[0]
    asset2 = pair[1]
    data_asset1 = np.array(df_meta_data.loc[asset1]).reshape(16, 1)
    data_asset2 = np.array(df_meta_data.loc[asset2]).reshape(16, 1)
    X_pair.append(np.array([data_asset1, data_asset2]))
    y.append(np.array([random.randint(0,1)]))

In [10]:
X_pair = np.array(X_pair).astype('float32')
y = np.array(y).astype('float32')

In [12]:
X_pair[0]

array([[[ 12.],
        [ 23.],
        [ 31.],
        [ 41.],
        [ 52.],
        [ 62.],
        [ 73.],
        [811.],
        [821.],
        [831.],
        [ 92.],
        [102.],
        [111.],
        [122.],
        [131.],
        [141.]],

       [[ 12.],
        [ 23.],
        [ 31.],
        [ 41.],
        [ 52.],
        [ 62.],
        [ 72.],
        [811.],
        [821.],
        [831.],
        [ 92.],
        [102.],
        [112.],
        [122.],
        [131.],
        [141.]]], dtype=float32)

In [11]:
def initializer_base_network_asset(input_shape):
    input = Input(shape = input_shape)
    x = Dense(128, activation = 'relu')(input)
    x = Dense(128, activation = 'relu')(x)
    x = Dense(64,activation = 'relu')(x)
    x = Dense(2,activation = 'relu')(x)
    return Model(inputs=input, outputs=x)

In [12]:
input_shape_ = (16, 1)
base_network = initializer_base_network_asset(input_shape_)

In [13]:
def euclid_dis(vects):
    x,y = vects
    sum_square = K.sum(K.square(x-y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [14]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


In [15]:
input_a = Input(shape=input_shape_)
input_b = Input(shape=input_shape_)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclid_dis,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

In [19]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([X_pair[:, 0], X_pair[:, 1]], y,
          batch_size=32,
          epochs=300)


Epoch 1/300
202/202 [==============================] - 2s 5ms/step - loss: 0.5044 - accuracy: 0.4955
Epoch 2/300
202/202 [==============================] - 1s 5ms/step - loss: 0.5044 - accuracy: 0.4959
Epoch 3/300
202/202 [==============================] - 1s 6ms/step - loss: 0.5044 - accuracy: 0.4947
Epoch 4/300
202/202 [==============================] - 1s 5ms/step - loss: 0.5044 - accuracy: 0.4955
Epoch 5/300
202/202 [==============================] - 1s 5ms/step - loss: 0.5044 - accuracy: 0.4959
Epoch 6/300
202/202 [==============================] - 1s 5ms/step - loss: 0.5044 - accuracy: 0.4951
Epoch 7/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4963
Epoch 8/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4955
Epoch 9/300
202/202 [==============================] - 1s 5ms/step - loss: 0.5044 - accuracy: 0.4955
Epoch 10/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accura

202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4955
Epoch 161/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4951
Epoch 162/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4951
Epoch 163/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4959
Epoch 164/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4959
Epoch 165/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4963
Epoch 166/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4951
Epoch 167/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4947
Epoch 168/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.4955
Epoch 169/300
202/202 [==============================] - 1s 4ms/step - loss: 0.5044 - a

In [20]:
# compute final accuracy on training and test sets
y_pred = model.predict([X_pair[:, 0], X_pair[:, 1]])
tr_acc = compute_accuracy(y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))

202/202 [==============================] - 1s 1ms/step
* Accuracy on training set: 49.53%
